In [1]:
# Load the neuroelectro data dump.  
import numpy as np
from sklearn.preprocessing import MultiLabelBinarizer,Imputer
import pandas
df = pandas.read_csv('http://dev.neuroelectro.org/static/src/article_ephys_metadata_curated.csv',delimiter='\t')

In [43]:
# define some python functions for working with the neurotree API

import urllib2
import json

def get_neurotree_node_id_from_pmid(pmid):
    query_url = 'http://neurotree.org/beta/include/check_pmid.php?term=%s' % pmid
    response = urllib2.urlopen(query_url)
    data = json.load(response)   
    if data:
        for author in data:
            if author['authorRank'] == 0:
                node_id = data[0]['pid']
                return node_id
        return None

def get_neurotree_node_info(neurotree_node_id):
    query_url = 'http://neurotree.org/neurotree/jsonQuery.php?querytype=node&pid=%s' % neurotree_node_id
    response = urllib2.urlopen(query_url)
    data = json.load(response)[0]
    return data
    
def get_investigator_path_len(neurotree_node_id_1, neurotree_node_id_2):
    if neurotree_node_id_1 == neurotree_node_id_2:
        return 0, None
    if neurotree_node_id_1 is None or neurotree_node_id_2 is None:
        return None, None
    query_url = 'http://neurotree.org/neurotree/distance.php?pid1=%s&pid2=%s&refresh=1&includera=1&includepd=1&includers=1&backonly=1&dispformat=json' %  (neurotree_node_id_1, neurotree_node_id_2)
    try:
        response = urllib2.urlopen(query_url)
    except:
        return None, None
    try:
        data = json.load(response)   
    except:
        return None, None
    print data
    if data:
        path_len = data['stepcount']
        common_inv = data['path1'][0]
        common_inv_name = get_neurotree_node_info(common_inv)['lastname']
        #print common_inv_name
    return path_len, common_inv_name
    

get_neurotree_node_info(4800)
node_id = get_neurotree_node_id_from_pmid(9185537)
get_investigator_path_len(77014, 4800)

{u'path2': [2715, 77014], u'pathcount': 2, u'stepcount': 2, u'path1': [2715, 4800], u'name2': u'Richard C Gerkin', u'name1': u'Shreejoy J Tripathy', u'pid1': 77014, u'pid2': 4800, u'backonly': u'1'}


(2, u'Urban')

In [29]:
neurotree_id_list = []

ca1_match = df['NeuronName'].isin(['Hippocampus CA1 pyramidal cell'])
pmid_list = list(set(df['Pmid'][ca1_match]))

for pmid in pmid_list:
    print pmid
    node_id = get_neurotree_node_id_from_pmid(pmid)
    neurotree_id_list.append(node_id)

9185537
[{u'authorRank': 3, u'authorList': u'Miller KK, Hoffer A, Svoboda KR, Lupica CR', u'pid': u'1157', u'value': u'0.08', u'label': u'KSVOBODA'}, {u'authorRank': 3, u'authorList': u'Miller KK, Hoffer A, Svoboda KR, Lupica CR', u'pid': u'4938', u'value': u'0.04', u'label': u'KSVOBODA'}, {u'authorRank': 0, u'authorList': u'Miller KK, Hoffer A, Svoboda KR, Lupica CR', u'pid': u'21681', u'value': u'0.01', u'label': u'CLUPICA'}]
17884930
[{u'authorRank': 0, u'authorList': u'Lamsa K, Irvine EE, Giese KP, Kullmann DM', u'pid': u'3677', u'value': u'1', u'label': u'DKULLMANN'}]
14645379
[{u'authorRank': 5, u'authorList': u'Li Q, Guo-Ross S, Lewis DV, Turner D, White AM, Wilson WA, Swartzwelder HS', u'pid': u'28831', u'value': u'0.04', u'label': u'AWHITE'}, {u'authorRank': 6, u'authorList': u'Li Q, Guo-Ross S, Lewis DV, Turner D, White AM, Wilson WA, Swartzwelder HS', u'pid': u'4108', u'value': u'0.36', u'label': u'WWILSON'}, {u'authorRank': None, u'authorList': u'Li Q, Guo-Ross S, Lewis DV,

In [31]:
print neurotree_id_list
unique_neurotree_nodes = sum(x is not None for x in neurotree_id_list)

unique_pubs_len = len(pmid_list)
unique_neurotree_nodes

print 'unique publications: %s' % unique_pubs_len
print 'found neurotree author ids: %s' % unique_neurotree_nodes

[u'1157', u'3677', None, u'2218', None, u'2613', u'18036', None, u'2223', None, u'9675', None, None, u'2492', None, u'1484', u'3776', u'13466', None, u'10162', u'81966', None, u'9675', None, u'458', u'5362', None, u'1645', None, u'50988', u'1645', None, u'900', u'64854', u'8474', None, u'29322', u'196', u'455', u'3776', None, None, u'9024', u'458', u'2701', u'2347', u'9675', u'1404', None, u'3355', u'4836', u'9675', None, u'1484', u'458', None, u'28752', u'7080', None, u'5067', u'2084', u'10080', None, None, u'8152', None, u'1484', None, u'3542', u'28749', u'2534', u'9675', u'1645', u'30514', u'5894', u'6909', None, u'4810', u'42508', u'5594', None]
unique publications: 81
found neurotree author ids: 55


In [26]:
get_investigator_path_len(neurotree_id_list[0], neurotree_id_list[5])

von Helmholtz


12

In [48]:
unique_neurotree_id_list = set(neurotree_id_list)
common_ancestor_list = []
path_len_list = []
for id1 in unique_neurotree_id_list:
    path_len_inner_list = []
    for id2 in unique_neurotree_id_list:
        print id1, id2
        #pathlen, common_ancestor = get_investigator_path_len(id1, id2)
        print pathlen, common_ancestor
        common_ancestor_list.append(common_ancestor)
        path_len_inner_list.append(pathlen)
    path_len_list.append(path_len_inner_list)

458 458
0 None
458 2218
0 None
458 2613
0 None
458 3776
0 None
458 None
0 None
458 30514
0 None
458 196
0 None
458 8152
0 None
458 10162
0 None
458 13466
0 None
458 9675
0 None
458 18036
0 None
458 1484
0 None
458 42508
0 None
458 4836
0 None
458 3677
0 None
458 6909
0 None
458 2347
0 None
458 10080
0 None
458 2223
0 None
458 28749
0 None
458 1404
0 None
458 1645
0 None
458 5067
0 None
458 1157
0 None
458 2701
0 None
458 455
0 None
458 3355
0 None
458 50988
0 None
458 81966
0 None
458 5894
0 None
458 8474
0 None
458 5362
0 None
458 7080
0 None
458 900
0 None
458 4810
0 None
458 5594
0 None
458 3542
0 None
458 2492
0 None
458 64854
0 None
458 2084
0 None
458 29322
0 None
458 28752
0 None
458 2534
0 None
458 9024
0 None
2218 458
0 None
2218 2218
0 None
2218 2613
0 None
2218 3776
0 None
2218 None
0 None
2218 30514
0 None
2218 196
0 None
2218 8152
0 None
2218 10162
0 None
2218 13466
0 None
2218 9675
0 None
2218 18036
0 None
2218 1484
0 None
2218 42508
0 None
2218 4836
0 None
2218 3677
0 No